**Knn_with_various_distance_measures**

In [1]:
#k-nearest neighbors on the Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import operator

# Load a data file
def load_data(filename):
    dataset = []
    with open(filename, 'r') as csvfile:
        filereader = reader(csvfile)
        for row in filereader:
            if not row:
                continue
            dataset.append(row)
    return dataset


# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup


# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = []
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = []
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split


# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = []
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = []
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
            predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy_measure = accuracy(actual, predicted)
        scores.append(accuracy_measure)
    return scores

# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += pow((row1[i] - row2[i]),2)
    return sqrt(distance)

#Calculate Hamming Distance
def hamming_distance(row1,row2):
    distance = 0.0
    for i in range(len(row1)-1):
        if row1[i]!=row2[i]:
            distance += 1
    return distance 

# Calculate the Chebyshev distance between two vectors
def chebyshev_distance(row1, row2):
    distance=[]
    for i in range(len(row1)-1):
        distance.append(abs(row1[i]-row2[i]))
    return max(distance)

# Calculate the Jaccard distance between two vectors
def jaccard_distance(row1, row2):
    intersection=len(set(row1).intersection(set(row2)))
    union= len(set(row1).union(set(row2)))
    return 1-(intersection / union)

# Calculate the Manhattan distance between two vectors
def manhattan_distance(row1, row2):
    distance=0
    for i in range(len(row1)-1):
        distance += sum([abs(row1[i]-row2[i])])
    return distance

# Calculate the Minkowski distance between two vectors
def minkowski_distance(row1, row2, p): 
    distance = 0.0
    for i in range(len(row1)-1):
        distance+= abs(row1[i]-row2[i])**p
    return distance

# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors, x):
    distances = list()
    p=2
    #print(x)
    
    for train_row in train:
        #Checking if input are the various distance measures
        if x == 'euclidean':
            dist = euclidean_distance(test_row, train_row)
        elif x== 'manhattan':
            dist = manhattan_distance(test_row, train_row) 
        elif x=='minkowski':
            dist = minkowski_distance(test_row, train_row,p) 
        elif x== 'hamming':
            dist = hamming_distance(test_row, train_row)
        elif x== 'chebyshev':
            dist = chebyshev_distance(test_row, train_row)
        elif x== 'jaccard':
            dist = jaccard_distance(test_row, train_row)
        else:
            print('error')
        
        distances.append((train_row, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

#to evaluate accuracy by calculating ratio of total correct predictions to all predictions 
def accuracy(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Make a prediction with neighbors
#to devise a predicted response based on most similar neighbours
#by having each neighbour vote for their class attribute and then taking max votes as the prediction

def predict_classification(train, test_row, num_neighbors,x):
    neighbors = get_neighbors(train, test_row, num_neighbors,x)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

# kNN Algorithm
def k_nearest_neighbors(train, test, num_neighbors, x):
    predictions = []
    for row in test:
        output = predict_classification(train, row, num_neighbors,x)
        predictions.append(output)
    return(predictions)

# Test the kNN on the dataset
seed(1)
x=input('Enter distance as "manhattan","minkowski","chebyshev","euclidean","hamming","jaccard": ')
filename = 'breast-cancer.data'#use dataset provided
dataset = load_data(filename)
for i in range(len(dataset[0])):
# convert class column to integers
    str_column_to_int(dataset, i)
# evaluate algorithm
n_folds = 10
num_neighbors = 5
scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors,x)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

#References:
#1.https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/
#2.https://machinelearningmastery.com/k-fold-cross-validation/
#3.https://machinelearningmastery.com/distance-measures-for-machine-learning/
#4.https://aiaspirant.com/distance-similarity-measures-in-machine-learning/

Enter distance as "manhattan","minkowski","chebyshev","euclidean","hamming","jaccard": manhattan
Scores: [78.57142857142857, 75.0, 60.71428571428571, 60.71428571428571, 67.85714285714286, 92.85714285714286, 89.28571428571429, 78.57142857142857, 85.71428571428571, 67.85714285714286]
Mean Accuracy: 75.714%
